In [36]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import os
import re
import pickle
import random
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import sys,fitz
import json

In [30]:
# Import CV and extract Data
nlp = spacy.blank('en')
stopwords = list(STOP_WORDS)
punctuation = punctuation + '\n'

train_data = pickle.load(open('train_data.pkl','rb'))
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [9]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
        
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotation in train_data:
                try:
                    nlp.update(
                        [text],
                        [annotation],
                        drop=0.2,
                        sgd=optimizer,
                        losses=losses)
                except Exception as e:
                    pass
                    
                        
            print(losses)  

In [10]:
train_model(train_data)

starting iteration 0
{'ner': 17697.081124774802}
starting iteration 1
{'ner': 11195.336227462998}
starting iteration 2
{'ner': 9442.964024490597}
starting iteration 3
{'ner': 8657.2391257055}
starting iteration 4
{'ner': 6901.92265736071}
starting iteration 5
{'ner': 5334.815422564294}
starting iteration 6
{'ner': 6754.890139539285}
starting iteration 7
{'ner': 5472.294728615241}
starting iteration 8
{'ner': 5685.78769064887}
starting iteration 9
{'ner': 6179.067870472694}


In [11]:
nlp.to_disk('nlp_model')
nlp_model = spacy.load('nlp_model')
train_data[0][0]
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')

NAME                          -Krishna Prasad
LOCATION                      -Patna
EMAIL ADDRESS                 -indeed.com/r/Krishna-Prasad/b8d7a1135a44a37a
LOCATION                      -Patna
DESIGNATION                   -Data Entry Operator
COMPANIES WORKED AT           -Microsoft
DEGREE                        -BSc in Computer Science
COLLEGE NAME                  -Magadh university
LOCATION                      -Patna


In [12]:
!pip install PyMuPDF

You should consider upgrading via the 'c:\users\akalanka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [14]:
fname = 'Test3.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
print(tx) 

Java Developer  Hyderabad, Telangana  To work in a firm with a professional work driven environment where I can utilize and apply   my knowledge, skills which would enable me as a fresh graduate to grow while fulfilling   organizational goals.      BASIC ACADEMIC CREDENTIALS      #readytowork  Willing to relocate: Anywhere  Work Experience  Java Developer  Kansas IT Solutions - Hyderabad, Telangana  June 2019 to July 2019  In an internship program I have successfully completed internship on Java Developer from  Kansas IT Solutions , Hyderabad .The duration of the project was from 3rd June to 3rd July .  Education  Bachelor's in Computer Science Engineering  Osmania University - Hyderabad, Telangana  June 2016 to June 2020  Higher Secondary(12th Pass) in Mathematics,Physics,Chemistry  Telangana Board of Secondary Education - Hyderabad, Telangana  June 2014 to June 2016    SKILLS  ● Python  ● Java  ● C  ● Web Development  ● Data Structure  ● Dbms  ● Html5  ● Programming  Projects / Paper

In [15]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')

NAME                          -Java Developer
LOCATION                      -Hyderabad
DESIGNATION                   -Java Developer
DEGREE                        -Bachelor's in Computer Science Engineering
COLLEGE NAME                  -Osmania University


In [18]:
docCV = nlp(tx)
word_frequenciesCV = {}
for word in docCV:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text.lower() not in word_frequenciesCV.keys():
                word_frequenciesCV[word.text.lower()] = 1
            else:
                word_frequenciesCV[word.text.lower()] += 1
            
print(word_frequenciesCV)

{'java': 4, 'developer': 3, ' ': 29, 'hyderabad': 5, 'telangana': 5, 'work': 3, 'firm': 1, 'professional': 1, 'driven': 1, 'environment': 1, 'utilize': 1, 'apply': 1, '  ': 2, 'knowledge': 1, 'skills': 2, 'enable': 1, 'fresh': 1, 'graduate': 1, 'grow': 1, 'fulfilling': 1, 'organizational': 1, 'goals': 1, '     ': 2, 'basic': 1, 'academic': 1, 'credentials': 1, 'readytowork': 1, 'willing': 1, 'relocate': 1, 'experience': 1, 'kansas': 2, 'solutions': 2, 'june': 6, '2019': 2, 'july': 2, 'internship': 2, 'program': 1, 'successfully': 1, 'completed': 1, '.the': 1, 'duration': 1, 'project': 3, '3rd': 2, 'education': 2, 'bachelor': 1, 'computer': 1, 'science': 1, 'engineering': 1, 'osmania': 1, 'university': 1, '2016': 2, '2020': 2, 'higher': 1, 'secondary(12th': 1, 'pass': 1, 'mathematics': 1, 'physics': 1, 'chemistry': 1, 'board': 1, 'secondary': 1, '2014': 1, '   ': 2, '●': 8, 'python': 2, 'c': 1, 'web': 1, 'development': 1, 'data': 1, 'structure': 1, 'dbms': 1, 'html5': 1, 'programming': 

In [25]:
payload = {'cvData': word_frequenciesCV}
response = requests.get("http://localhost:8080/calculateSimilarity" ,params=payload)
print(response.status_code)

200


In [ ]:
####################################################################
####################################################################

In [26]:
#Get Term List From Job Descriptions
class Job:
    def __init__(self, URI, description , summary):
        self.URI = URI
        self.description = description
        self.summary = summary
    def __iter__(self):
        return self
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)

In [32]:
page_list = [10]

jobDic = {}
i=0
jobList = []

jobs_categories = ['java+developer']
with open('G:/Research/ScrapingDataFromIndeed.com/job_description.csv','a',encoding='utf-8',newline='') as f_output:
    csv_print = csv.writer(f_output)
    
    file_is_empty = os.stat('G:/Research/ScrapingDataFromIndeed.com/job_description.csv').st_size== 0
    if file_is_empty:
        csv_print.writerow(['Job_Title','Company','Location','Summary','Jobs_Description','Salary'])
    
    for job_list in jobs_categories:
        
        for page in page_list:
            source = requests.get('https://www.indeed.com/jobs?q='+job_list+'&start={}'.format(page)).text

            soup = BeautifulSoup(source,'lxml')
            
            for jobs in soup.find_all(class_='result'):
                
                
                print("-------------------------------------------")
                requests.get('https://www.indeed.com/jobs?q='+job_list+'&start={}'.format(page)).text

                try:
                    
                    getJK = jobs.find(class_='title') 
                    getJK1 = getJK.find('a')
                    #print(getJK1)
                    getJK2 = re.search('"jl_(.+?)"',str(getJK1)).group(1)
                    #print(getJK2)
               
                    links = soup.find_all('script')
                    links2 = str(links)
                    word = re.search('tk (.*)',links2)
                    word2 =  str(word.groups())
                    match = re.search(r'["](.*?)["]',word2).group()
                    #print( match )

                    find_jk = soup.find(class_='result')
                    find_jk1 = find_jk.find('a')
                    find_jk2 = str(find_jk1)
                    find_jk3 = re.search(r'jk=(.*?)&',find_jk2).group(1)
                    #print(find_jk3)
                    
                    #source1 = requests.get('https://www.indeed.com/viewjob?'+find_jk3+''+match+'&from=serp&vjs=3').text

                    source1 = requests.get('https://www.indeed.com/viewjob?jk='+getJK2+'&'+match+'&from=serp&vjs=3').text

                    soup1 = BeautifulSoup(source1,'lxml')
                    jobs_description = soup1.find(class_='jobsearch-jobDescriptionText').text.strip()
                    #print('*************************')
                    print(jobs_description)
                    
                except Exception as e:
                    jobs_description = None

                try:
                    job_title = jobs.find(class_='title').text.strip()
                except Exception as e:
                    job_title = None
                print('Job_Title' ,job_title)

                try:
                    company_name = jobs.find('span',class_='company').text.strip()
                except Exception as e:
                    company_name = None
                #print('Company_Name' ,company_name)

                try:
                    company_location = jobs.find('span',class_='location').text.strip()
                except Exception as e:
                    company_location = None
                #print('Company_Location' ,company_location)

                try:
                    summary = jobs.find(class_='summary').text.strip()
                except Exception as e:
                    summary = None
                #print('Summary' ,summary)

                try:
                    salary = jobs.find(class_='salary').text.strip()
                except Exception as e:
                    salary = None
                #print('Salary' ,salary)
                
#                 jobDic[i] = {'URI' : job_title ,'description' : company_location}
#                 i = i+1
                
                s = Job('https://www.indeed.com/viewjob?jk='+getJK2+'&'+match+'&from=serp&vjs=3', company_location , jobs_description)
                jobList.append(s)

                csv_print.writerow([job_title,company_name,company_location,summary,jobs_description,salary])

                time.sleep(2)
                
        #print(jobDic['description'])
                

-------------------------------------------
JOB DESCRIPTION
Skill: Java Developer
Role: T3, T2
Interested in a full cycle development role working collaboratively with a diverse suite of clients and a team that cares about your growth? Join Virtusa!
As a Java Developer, you will have the opportunity to tap into your knowledge to develop, design and manage a wide range of innovative projects. Sound like fun? Read on.

Day-to-day responsibilities will include:
 Utilizing your deep technical knowledge to design, develop and optimize business processes across multiple systems. Developing detailed technical analysis and design specifications based on functional requirements.
 Collaborating with fellow developers and clients to solve problems.
 Creating user acceptance testing and performance testing plans to validate and support the customizations, features and functionalities.
 Migrating to internal and external clouds, as well as to Microservices architecture.
 Evaluating open source and 

-------------------------------------------
Responsibilities

Java developer position will help analyze user needs and define site objectives. They will use programming languages and software to build applications. Design and implement efficient Ur’s for internet and intranet applications. Work together with other developers to produce optimal outcomes.

Experience required : 1 year

Number of posts : 1
Required Languages
Java
Python
Spring Boot
Following Degrees
Computer Science
Information Technology
Job_Title JR. Java Developer
-------------------------------------------
Comments:

This is a contract to hire role.
Pittsburgh location only
Will require a coding session during interview
Will work remote to start then must be on site in Pittsburgh once restrictions are lifted
Full time conversion salary is $80K to 85K +/ . MIGHT be able to accommodate $90 K.
Video technical interview required. This is a hands on very technical role.

Description:

Member of an agile software engineer t

-------------------------------------------
Location: Remote
Requirements:

Java/J2EE and various technologies in this ecosystem (JSP, Spring, JDBC, Hibernate, Maven, Tomcat, etc.),
Web based application development experience with HTML/HTML5, Javascript and frameworks (jquery, prototype, etc.)


Nice to have:

Knows how to write web services (SOAP, REST, AJAX, etc),
Experience with database systems (SQL/RDBMS, NoSQL),
Scripting languages: shell, perl or python,
Documentation skills
Crypto, SSL/TLS, PKI,
Network protocols on user and programming level,
Secure coding practices,
Development experience with distributed systems,
Good English skills,
Masters degree in Information Technology or similar.
When applying please send us the following to jobs@nxlog.org:
a short cover letter explaining why you are interested in the position,
your CV/resume,
salary requirements (hourly rate).
Job_Title Java Developer
-------------------------------------------
Required Skills /Qualifications: • Stro

In [33]:
for s in jobList:
    print(s.description)

Fremont, CA
Herndon, VA
Jersey City, NJ 07306 (Hackensack Riverfront area)
Pittsburgh, PA
Raleigh, NC
Dallas, TX
Pittsburgh, PA
Arlington, VA
Ann Arbor, MI 48104
Chicago, IL
Hoboken, NJ
United States
Tampa, FL
Chicago, IL 60606 (Loop area)
United States


In [34]:
word_frequencies = {}
new_word_frequencies = []
finalJobDescriptionList = {}
jobListFinal = []

for s in jobList:
    word_frequencies = {}
    new_word_frequencies = []
    text = str(s.summary)
    URI =str(s.URI)
    doc = nlp(text)
    tokens = [token.text for token in doc]
    
    for word in doc:
             if word.text.lower() not in stopwords:
                if word.text.lower() not in punctuation:
                    if word.text.lower() not in word_frequencies.keys():
                        word_frequencies[word.text.lower()] = 1
                        new_word_frequencies.append(word.text.lower())
                    else:
                        word_frequencies[word.text.lower()] += 1
                        
    s = Job(URI,word_frequencies,new_word_frequencies)
    jobListFinal.append(s)
    print(new_word_frequencies)

['job', 'description', 'skill', 'java', 'developer', 'role', 't3', 't2', 'interested', 'cycle', 'development', 'working', 'collaboratively', 'diverse', 'suite', 'clients', 'team', 'cares', 'growth', 'join', 'virtusa', 'opportunity', 'tap', 'knowledge', 'develop', 'design', 'manage', 'wide', 'range', 'innovative', 'projects', 'sound', 'like', 'fun', 'read', 'day', 'responsibilities', 'include', '\n ', 'utilizing', 'deep', 'technical', 'optimize', 'business', 'processes', 'multiple', 'systems', 'developing', 'detailed', 'analysis', 'specifications', 'based', 'functional', 'requirements', 'collaborating', 'fellow', 'developers', 'solve', 'problems', 'creating', 'user', 'acceptance', 'testing', 'performance', 'plans', 'validate', 'support', 'customizations', 'features', 'functionalities', 'migrating', 'internal', 'external', 'clouds', 'microservices', 'architecture', 'evaluating', 'open', 'source', 'vendor', 'products', 'qualifications', 'bachelor', 'degree', 'computer', 'science', 'engine

In [83]:
json_string = json.dumps([ob.__dict__ for ob in jobListFinal])
print(json_string)

payload = {'cvData': word_frequenciesCV,'jobDataList': json_string}

# response = requests.get("http://localhost:8080/getJobs",params=payload)
# print(response.status_code)

response = requests.get("http://localhost:8080/calculateSimilarity",params=payload)
print(response.status_code)
                

[{"URI": "https://www.indeed.com/viewjob?jk=6cab4f2b8ace4a0e&\"1ecka0or5nvck800\"&from=serp&vjs=3", "description": {"job": 3, "description": 1, "skill": 1, "java": 3, "developer": 2, "role": 2, "t3": 1, "t2": 1, "interested": 1, "cycle": 1, "development": 4, "working": 3, "collaboratively": 1, "diverse": 1, "suite": 1, "clients": 2, "team": 4, "cares": 2, "growth": 2, "join": 3, "virtusa": 5, "opportunity": 2, "tap": 1, "knowledge": 3, "develop": 2, "design": 5, "manage": 1, "wide": 1, "range": 1, "innovative": 1, "projects": 2, "sound": 1, "like": 1, "fun": 1, "read": 1, "day": 2, "responsibilities": 1, "include": 1, "\n ": 24, "utilizing": 1, "deep": 1, "technical": 2, "optimize": 1, "business": 2, "processes": 1, "multiple": 2, "systems": 3, "developing": 1, "detailed": 1, "analysis": 1, "specifications": 1, "based": 1, "functional": 2, "requirements": 2, "collaborating": 1, "fellow": 1, "developers": 1, "solve": 1, "problems": 1, "creating": 1, "user": 1, "acceptance": 1, "testing"

500
